# What's New in Mojo 24.2

<img src="whats_new.jpeg" alt="Drawing" style="width: 800px;"/>

## Core language features

In [ ]:
from random import rand
struct MojoArray[dtype: DType = DType.float64]():
    var _ptr: DTypePointer[dtype]
    var numel: Int

    fn __init__(inout self, numel: Int, *data: Scalar[dtype]):
        self.numel = len(data)
        self._ptr = DTypePointer[dtype].alloc(len(data))
        for i in range(len(data)):
            self._ptr[i] = data[i]

    fn __len__(self) -> Int:
        return self.numel

    fn __str__(self) -> String:
        var s: String = ""
        s += "["
        for i in range(len(self)):
            if i>0:
                s+=" "
            s+=self._ptr[i]
        s+="]"
        return s

var arr = MojoArray[DType.index](1,2,3,4,5,6,7)
print("Array and length:")
print(arr)
print("Length: ",len(arr))

In [ ]:
from math.polynomial import polynomial_evaluate
alias dtype = DType.float64

def polynomial(x: Float64, **kwargs: Int):
    var val: Float64 = 0.0
    var idx: Int = 0
    var s: String = ""
    for key in kwargs.keys():
        if idx == 0:
            s += key[]+" + "
        else:
            s+= key[]+"*x^"+str(idx)+" + "
        val += kwargs[key[]]*(x**idx)
        idx+=1

    print("Evaluating polynomial: [",s[:-2],'] ',"at",x)
    print(val)

polynomial(42.0, a=-6, b=3, c=-2, d=1)
alias coeff = List[Scalar[dtype]](-6, 3, -2, 1)
print("Compare results with math.polynomial")
print(polynomial_evaluate[dtype, 1, coeff](42.0))

## Standard library features

#### Mojo Lists

In [ ]:
from random import random_si64

def stack_sort(list: List[Int]) -> List[Int]:
    stack = List[Int]()
    input_list = List[Int](list)  # Create a copy of the input list
    sorted_list = List[Int]()  # This will hold the sorted elements

    while len(input_list):
        temp = input_list.pop_back()
        while len(stack) and stack[-1] < temp:
            input_list.append(stack.pop_back())
        stack.append(temp)

    while len(stack):
        sorted_list.append(stack.pop_back())
    return sorted_list

def print_list(list: List[Int]):
    print('List','[', sep=': ', end=' ')
    for i in range(len(list)):
        print(list[i],end=' ')
    print("]", end='\n')

my_list = List[Int]()
for i in range(5):
    my_list.append(int(random_si64(0,100)))

print("Original")
print_list(my_list)
sorted_list = stack_sort(my_list)
print("Sorted")
print_list(sorted_list)

In [ ]:
from tensor import Tensor, rand
var t1 = rand[DType.float32](2,2)
var t2 = rand[DType.float32](3,2)
var t3 = rand[DType.float32](1)

var lst = List[Tensor[DType.float32]](t1,t2,t3)
for i in range(len(lst)):
    print(lst[i])

#### Tensor `argmax()` and `argmin()`

In [ ]:
from tensor import Tensor, rand, TensorShape
var t = rand[DType.float32](3,100)
print(t.argmax().astype[DType.index]())
print(t.argmax(axis=-1).astype[DType.index]())

#### `math.ulp()`

In [ ]:
from math.polynomial import polynomial_evaluate
from math import abs, max, ulp
alias dtype = DType.float64
alias width = simdwidthof[dtype]()

alias coeff = List[Scalar[dtype]](-6, 3, -2, 1)
alias deriv_coeff = List[Scalar[dtype]](3, -4, 3)

def f(x: Float64) -> Float64:
    return polynomial_evaluate[dtype, 1, coeff](x)

def df(x: Float64) -> Float64:
    return polynomial_evaluate[dtype, 1, deriv_coeff](x)

def newton_raphson(x0: Float64, tolerance: Float64) -> Float64:
    var x = x0
    var steps: Int = 0
    while True:
        x_new = x - f(x)/df(x)
        steps += 1
        if abs(x_new - x) < max(tolerance, ulp(x_new)):
            print("Steps to convergence:", steps)
            break
        x = x_new
    return x_new

initial_guess = 100  # Initial guess for the root
tolerance = 1e-2    # Tolerance for terminating the algorithm

root = newton_raphson(initial_guess, tolerance)
print("Root found using Newton-Raphson method:", root)
print("Value of the polynomial at the root:", polynomial_evaluate[dtype, 1, coeff](root))

## Python Interoperability

In [ ]:
from python import Python
from math.polynomial import polynomial_evaluate

var np = Python.import_module("numpy")
var plt = Python.import_module("matplotlib.pyplot")

alias dtype = DType.float64
alias width = simdwidthof[dtype]()
alias coeff = List[Scalar[dtype]](-6, 3, -2, 1)
alias deriv_coeff = List[Scalar[dtype]](3, -4, 3)

x_vals = np.linspace(-2, 3, 100)
var y_vals: PythonObject = []
var y_deriv_vals: PythonObject = []

for i in range(len(x_vals)):
    y_vals.append(polynomial_evaluate[dtype, 1, coeff](x_vals[i].to_float64()))
    y_deriv_vals.append(polynomial_evaluate[dtype, 1, deriv_coeff](x_vals[i].to_float64()))

plt.figure(figsize=(10, 6))
plt.plot(x_vals, y_vals, label='Polynomial: $x^3 - 2x^2 + 3x - 6$')
plt.plot(x_vals, y_deriv_vals, label='Derivative of the Polynomial')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Plot showing root of $x^3 - 2x^2 + 3x - 6$')
plt.legend()
plt.grid(True)
plt.axhline(0, color='black',linewidth=0.5)
plt.axvline(0, color='black',linewidth=0.5)
plt.axvline(x=2, color='r', linestyle='--', linewidth=2)

plt.plot(2, 0, marker='o', markersize=15, color='b')
plt.text(2.1, -2, 'Root', fontsize=12, color='b')

plt.show()